In [1]:
from neo4j import GraphDatabase, Query, Record
from neo4j.exceptions import ServiceUnavailable, SessionError

In [4]:
uri = "bolt://83.229.84.12:7687"  
username = "tumaiReadonly"
password = "MAKEATHON2024"  
database = "graph2.db"
AUTH = (username, password)

driver = GraphDatabase.driver(uri, auth=AUTH)

In [17]:
with driver.session(database=database) as session:
	data = session.run("MATCH (n:Biological_sample) RETURN n.subjectid AS id")
	ids = [r["id"] for r in data]

In [19]:
ids[0]

'10006'

"\n        MATCH (b:Biological_sample {subjectid: '10006'})-[:HAS_DISEASE]->(d:Disease)\n        WHERE NOT d.name = 'control'\n        OPTIONAL MATCH (b)-[:HAS_PROTEIN]->(p:Protein)\n        OPTIONAL MATCH (b)-[:HAS_PHENOTYPE]->(ph:Phenotype)\n        WITH b, \n            collect(DISTINCT p.id) AS proteins,\n            collect(DISTINCT ph.id) AS phenotypes,\n            d.synonyms AS synonyms\n        UNWIND synonyms AS synonym\n        WITH b, proteins, phenotypes, synonym\n        WHERE synonym CONTAINS 'ICD10CM:'\n        RETURN b.subjectid AS subject_id, \n            proteins,\n            phenotypes,\n            substring(synonym, size('ICD10CM:'), 1) AS disease\n    "

In [23]:
with driver.session(database=database) as session:
	data = session.run(f"""
        MATCH (b:Biological_sample {{subjectid: '{ids[0]}'}})-[:HAS_DISEASE]->(d:Disease)
        WHERE NOT d.name = 'control'
        OPTIONAL MATCH (b)-[:HAS_PROTEIN]->(p:Protein)
        OPTIONAL MATCH (b)-[:HAS_PHENOTYPE]->(ph:Phenotype)
        WITH b, 
            collect(DISTINCT p.id) AS proteins,
            collect(DISTINCT ph.id) AS phenotypes,
            d.synonyms AS synonyms
        UNWIND synonyms AS synonym
        WITH b, proteins, phenotypes, synonym
        WHERE synonym CONTAINS 'ICD10CM:'
        RETURN b.subjectid AS subject_id, 
            proteins,
            phenotypes,
            substring(synonym, size('ICD10CM:'), 1) AS disease
    """)
	print([*data.data()])

[{'subject_id': '10006', 'proteins': ['A0A493QPB3', 'Q96MI9', 'B0YJ93', 'A0A023I7H5', 'A0A059RY54', 'B7ZLZ9', 'A0A3S6RHL8', 'Q8NEF3', 'F5H3I8', 'A0A0C4DGS9', 'Q9BU19-4', 'A0A5C2GVB0', 'A0A1W2PR96', 'A0A5C2FWW4', 'Q07826', 'Q6LBK2', 'A0A5C2GUP6', 'O60293-1', 'A0A286YEY4', 'A8TX70-2', 'I3Y3J5', 'A0A5C2G1M9', 'A0A087WYF3', 'A0A288Q9R2', 'Q9NXF8', 'O43886', 'F8VWW8', 'E9LT22', 'H2DML0', 'S4R369', 'Q6LC06', 'O15533-2', 'S6BGE5', 'Q08EJ9', 'Q8NH37', 'Q8IZV2', 'Q8IZA9', 'H7BZB6', 'Q6RJW4', 'Q6WQV4'], 'phenotypes': ['HP:0032369', 'HP:0025066', 'HP:0001873', 'HP:0003236', 'HP:0002902', 'HP:0001899', 'HP:0002918', 'HP:0003228', 'HP:0003072', 'HP:0020059', 'HP:0001923', 'HP:0012417', 'HP:0031963', 'HP:0012416', 'HP:0011031', 'HP:0020063', 'HP:0002907', 'HP:0020062', 'HP:0002151', 'HP:0002905', 'HP:0031962', 'HP:0011899', 'HP:0008151', 'HP:0031969', 'HP:0002148', 'HP:0003645', 'HP:0011900', 'HP:0008163', 'HP:0003281', 'HP:0032198', 'HP:0002901', 'HP:0032437', 'HP:0003074', 'HP:0410174', 'HP:000194